### QNodeCollection

In [1]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 20

dev = qml.device('lightning.qubit', wires=n_qubits)

In [3]:
@qml.qnode(dev)
def circuit():
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    return qml.expval(qml.PauliZ(0))

# one circuit execution
%time circuit() 

CPU times: user 33 ms, sys: 10.5 ms, total: 43.5 ms
Wall time: 40.8 ms


tensor(0., requires_grad=True)

In [10]:
circuits_count = 50

circuits = qml.QNodeCollection([circuit] * circuits_count)

In [11]:
# two circuit executions
%time circuits(parallel=True)

CPU times: user 1.62 s, sys: 66.3 ms, total: 1.69 s
Wall time: 1.5 s


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
from braket.tracking import Tracker

cost_tracker = Tracker().start()

qubits = list(range(n_qubits))

sv1_batched = qml.device(
    name="braket.aws.qubit",
    device_arn="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    wires=qubits,
    shots=1000,
    parallel=True,
)

In [34]:
@qml.qnode(sv1_batched)
def sv1_batched_circuit():
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    return qml.expval(qml.PauliZ(0))

In [24]:
# one circuit execution
%time sv1_batched_circuit() 

CPU times: user 270 ms, sys: 0 ns, total: 270 ms
Wall time: 8.09 s


tensor(-0.038, requires_grad=True)

In [25]:
cost_tracker.quantum_tasks_statistics()

{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 1000,
  'tasks': {'COMPLETED': 1},
  'execution_duration': datetime.timedelta(microseconds=25000),
  'billed_execution_duration': datetime.timedelta(seconds=3)}}

In [35]:
circuits_count = 3

sv1_batched_circuits = qml.QNodeCollection([sv1_batched_circuit] * circuits_count)

In [37]:
sv1_batched_circuits()

array([ 0.006, -0.006,  0.022])

In [38]:
cost_tracker.quantum_tasks_statistics()

{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 3000,
  'tasks': {'COMPLETED': 3},
  'execution_duration': datetime.timedelta(microseconds=177000),
  'billed_execution_duration': datetime.timedelta(seconds=9)}}